# Get More Data 
expand time horizon Jan 1 2019 - July 31, 2020 - 5 quarters. note 45 day (safe: 2 mo.) window from now (10/08/2020)

In [1]:
# install.packages("readr")

In [1]:
library(bigrquery)
library(plotly)
library(data.table)
library(stringr)
library(readr)

Loading required package: ggplot2


Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:stats’:

    filter


The following object is masked from ‘package:graphics’:

    layout




In [2]:
objects()

character(0)

In [3]:
projectid <- "radixbi-249015"

## count number of rows we're working with

In [11]:
expiry_date_start='2018-01-01'
expiry_date_end='2020-07-31'

source("../phaseII_06_fallbacksupp/query_string_defs_2.R")

# writeLines(count_expiry_data_str)

In [12]:
sql <- count_expiry_data_str

bq_table_download(bq_project_query(projectid, sql))

f0_
<int>
6524


In [ ]:
# webUI returns 9606009 rows -- WHY??

## compare to size of first_renewal_preds

dim = (1949082      38) <br>
size = 807.4 Mb = 0.8074 Gb

In [3]:
# first_renewal_preds <- readRDS('../../data/first_renewal_preds')
# print(dim(first_renewal_preds))
# print(object.size(first_renewal_preds), units = "auto") 
# rm(first_renewal_preds)

[1] 1949082      38
807.4 Mb


In [23]:
6200015/1949082

[1] 3.180992

## size is almost 320% larger (row count) than first_renewal_preds. <br>will take 2.6 Gigabytes

In [26]:
projectid <- "radixbi-249015"
expiry_date_start <- '2020-10-06'
expiry_date_end <- '2020-10-10'

source("query_string_defs_2.R")

sql <- get_expiry_data_str

expiry_data <- bq_table_download(bq_project_query(projectid, sql))
print(dim(expiry_data))
print(object.size(expiry_data), units = "auto") 

# [1] 1743356      30
# 475.9 Mb

[1] 64821    27
18.2 Mb


## qa rest of get_expiry_data.R code

In [12]:
expiry_data_orig <- expiry_data

In [13]:
names(expiry_data_orig)

[1] "renewal_type"              "renewed_count"            
 [3] "Expiry_date"               "domain_id"                
 [5] "domain"                    "creation_date1"           
 [7] "status"                    "tld"                      
 [9] "registrar_shortname"       "client_shortname"         
[11] "client_country"            "registrant_country"       
[13] "region"                    "noofyears"                
[15] "renewal_status"            "renew_mbg"                
[17] "renewal_item_id"           "renew_type"               
[19] "autorenew_type"            "renew_date"               
[21] "Renew_Registrar_Shortname" "Renew_Client_Shortname"   
[23] "domain_revenue"            "arpt"                     
[25] "gross_profit"              "gp_less_icann_fixed"      
[27] "renew_domain_years"        "renew_domain_revenue"     
[29] "renew_arpt"                "registration_arpt"

In [14]:
expiry_data < -expiry_data_orig %>%
select(renewal_type, renewed_count, expiry_date = Expiry_date, domain_id, domain,
       creation_date = creation_date1, status, tld, registrar = registrar_shortname,
       reseller = client_shortname, reseller_country = client_country, region, reg_period = noofyears,
       registrant_country,renewal_status,renew_mbg, renew_type, autorenew_type, renew_date, renew_registrar = Renew_Registrar_Shortname, 
       renew_reseller = Renew_Client_Shortname, reg_revenue = domain_revenue, reg_arpt = arpt,
       renew_period = renew_domain_years, renew_domain_revenue, renew_arpt,reg_arpt_org = registration_arpt)

In [16]:
names(expiry_data)

[1] "renewal_type"         "renewed_count"        "expiry_date"         
 [4] "domain_id"            "domain"               "creation_date"       
 [7] "status"               "tld"                  "registrar"           
[10] "reseller"             "reseller_country"     "region"              
[13] "reg_period"           "registrant_country"   "renewal_status"      
[16] "renew_mbg"            "renew_type"           "autorenew_type"      
[19] "renew_date"           "renew_registrar"      "renew_reseller"      
[22] "reg_revenue"          "reg_arpt"             "renew_period"        
[25] "renew_domain_revenue" "renew_arpt"           "reg_arpt_org"

In [17]:
expiry_data$expiry_date<-as.Date(expiry_data$expiry_date, "%Y-%m-%d")
expiry_data$creation_date<-as.Date(expiry_data$creation_date, "%Y-%m-%d")
expiry_data$renew_date<-as.Date(expiry_data$renew_date, "%Y-%m-%d")
expiry_data$registrar<-tolower(expiry_data$registrar)
expiry_data$reseller<-tolower(expiry_data$reseller)

In [18]:
expiry_data$renewal_status[!(expiry_data$renew_mbg == 0)]<-"Not Renewd"
expiry_data$autorenew_type[!(expiry_data$renew_mbg == 0)]<-"unrealized"

In [19]:
saveRDS(expiry_data,"../../data/output/expiry_data")
system("gsutil cp /home/jupyter/local/Domains_202003/data/output/* gs://data_outputt/output/")

# expiry_data <- readRDS(file.path('../../data/output/',"expiry_data"))